In [27]:
import numpy as np
import xml.etree.ElementTree as ET
import pyproj

In [28]:
def mat2r_t(mat1, mat2):
    U, S, Vt = np.linalg.svd(np.dot(mat2.T, mat1))
    R = np.dot(U, Vt)
    if np.linalg.det(R) < 0:
        R[:, 2] *= -1
    org = np.dot(np.linalg.inv(mat2),mat1)
    t = org[:3,3]/np.linalg.norm(org[:3,3])
    return R, t

In [50]:
class CameraRef:
    def __init__(self, camera_xml, calib_xml_opencv):
        """Input takes:
        1. camera_xml file from: metashape->File->Export->Export Cameras
        2. calib_xml file from: metashape->Tools->Camera Calibration->Adjusted->Save As(OpenCv Format)
        """
        self.root = ET.parse(camera_xml).getroot()
        self.camera_xml = camera_xml
        self.calib_xml_opencv = calib_xml_opencv
        self.camera_calibration_mat = self.get_camera_calib()
        self.camera_calibration_mat_opencv = self.get_camera_calib_opencv()
        self.camera_dict = self.get_cam_dict()
        self.local2global = self.get_local2global_matrix()

    def get_local2global_matrix(self):
        root = self.root
        rotation = np.matrix(root[0][3][0].text).reshape(3,3)
        translation = np.matrix(root[0][3][1].text).reshape(1,3)
        scale = float(root[0][3][2].text)
        rt = np.vstack([np.hstack([scale*rotation,translation.T]),[0,0,0,1]])
        return rt

    def get_cam_dict(self):
        root = self.root
        camera = []
        for cam in root.iter("camera"):
            camera_id = int(list(cam.attrib.values())[0])
            img_id = list(cam.attrib.values())[-1]
            #print(camera_id, img_id)
            for item in cam.iter("transform"):
                val = np.matrix(item.text)
                transform_mat = np.reshape(val, (4,4))
                #print(transform_mat)
            for item in cam.iter("reference"):
                ref = item.attrib
                #print(ref)
            camera_dict = {"image_id" : img_id,
                                "transform_mat" : transform_mat,
                                "reference" : ref}
            camera.append(camera_dict)
        return camera

    def get_camera_calib(self):
        root = self.root
        camera_intrinsics = []
        y = []
        for node in root.iter("calibration"):
            x = list(node.attrib.values())
            if (x[1]=="adjusted"):
                for node2 in node.iter():
                    camera_intrinsics.append(node2.text)
                    y.append(list(node2.attrib.values()))
        camera_intrinsics = list(map(float,camera_intrinsics[2:]))
        width, height = list(map(float,y[1]))
        camera_intrinsics_dict = {"width":width,
                                  "height":height,
                                  "f":camera_intrinsics[0],
                                  "cx":camera_intrinsics[1],
                                  "cy":camera_intrinsics[2],
                                  "b1":camera_intrinsics[3],
                                  "b2":camera_intrinsics[4],
                                  "k1":camera_intrinsics[5],
                                  "k2":camera_intrinsics[6],
                                  "k3":camera_intrinsics[7],
                                  "k4":camera_intrinsics[8],
                                  "p1":camera_intrinsics[9],
                                  "p2":camera_intrinsics[10]}
        return camera_intrinsics_dict
    def get_camera_calib_opencv(self):
        tree = ET.parse(self.calib_xml_opencv)
        root = tree.getroot()
        for x in  root.iter("Camera_Matrix"):
            for y in x.iter("data"):
                camera_mat = np.matrix(y.text)
        for x in  root.iter("Distortion_Coefficients"):
            for y in x.iter("data"):
                distort_mat = np.matrix(y.text)
        camera_mat = np.asarray(camera_mat.reshape((3,3)))
        distort_mat = np.asarray(distort_mat)
        #print(camera_mat, distort_mat)
        return [camera_mat, distort_mat]
class Camera:
    """
    Camera class inputs:
    1. Camera Dict
    2. local2global transformation mat from CameraRef Class
    In stores:
    1. Camera Id/Image name
    2. Pose matrix in internal metashape coord
    3. reference on global coord
    4. method for Pose in global coord
    """
    def __init__(self, camera_dict, local2global):
        self.camera_id = camera_dict["image_id"]
        self.pose = camera_dict["transform_mat"]
        self.reference = camera_dict["reference"]
        self.local2global_transmat =local2global

    def get_global_pose(self):
        posecrs4978 = self.local2global_transmat*self.pose
        crs4978 = CRS.from_epsg(4978)
        crs32755 = CRS.from_epsg(32755)
        proj = Transformer.from_crs(crs4978, crs32755)
        posecrs4978[0,3], posecrs4978[1,3], posecrs4978[2,3] = proj.transform(posecrs4978[0,3], posecrs4978[1,3], posecrs4978[2,3])
        posecrs32755 = posecrs4978
        return posecrs32755

In [51]:
x = CameraRef("/home/turin/Desktop/lizard_dataset_curated/2014/cam14.xml","/home/turin/Desktop/lizard_dataset_curated/opencv_cam_calib.xml" )
rt =x.local2global
cam1 = Camera(x.camera_dict[0], rt)
cam1pose = cam1.get_global_pose()
cam1pose
# #crs = CRS.from_epsg(4326)
# crs4978 = CRS.from_epsg(4978)
# crs32755 = CRS.from_epsg(32755)
# proj = Transformer.from_crs(crs4978, crs32755)
# wgs32755 = proj.transform(cam1pose[0,3], cam1pose[1,3], cam1pose[2,3])
# wgs32755

matrix([[-2.59752374e-02, -5.60338363e-03,  5.55976310e-02,
          3.32448523e+05],
        [-2.98213292e-02, -5.04599522e-02, -1.90181228e-02,
          8.37565686e+06],
        [ 4.72565637e-02, -3.49228115e-02,  1.85586064e-02,
          9.49451431e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]])

In [33]:
from pyproj import CRS
from pyproj import Transformer
crs = CRS.from_epsg(32755)
crs2 = CRS.from_epsg(4326)
proj = Transformer.from_crs(crs, crs.geodetic_crs)
proj.transform(cam1pose[:3,3].T)

TypeError: transform() missing 1 required positional argument: 'yy'

In [44]:
cam1pose[1,3]

3500298.0164774703

In [46]:
wgs32755

(332448.52323560254, 8375656.862874637, 0.9494514325633645)